In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import pygplates
import pygmt
import xarray as xr
import seaborn as sns

from scipy.stats import median_abs_deviation

from gprm.datasets import Rocks, Reconstructions, Paleogeography
from gprm.utils.raster import to_anchor_plate

import sys
#sys.path.append('/Users/simon/OneDrive/Andes_works//python/')
sys.path.append('../python/')
import joyful_geochemistry as joy
import joyful_mapping as joymap
#import time_series as ts

%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib as mpl
mpl.rc('font',family='Helvetica')
mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams['xtick.major.width'] = 2
mpl.rcParams['ytick.major.width'] = 2

import warnings
#from pandas.core.common import SettingWithCopyWarning
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.filterwarnings(action='ignore', message='All-NaN axis encountered')


#Paleomap = Reconstructions.fetch_Scotese()
#PaleoDEM = Paleogeography.fetch_Paleomap()
import collections
from gprm import ReconstructionModel

boschman_rotation_model = ReconstructionModel('')
boschman_rotation_model.add_rotation_model('/Users/simon/GIT/bx/andes//boschman/reconstruction_model/boschman_reverse_engineered_rotations.rot')
boschman_rotation_model.add_static_polygons('/Users/simon/GIT/bx/andes//boschman/reconstruction_model/reconstructed_0.00Ma.shp')

raster_dict = {}
for reconstruction_time in np.arange(0,81,1):
    raster_dict[reconstruction_time] = '/Users/simon/GIT/bx/andes//boschman/grids/boschman_DEM_{:0.0f}Ma.nc'.format(reconstruction_time)

boschman_rasters = collections.OrderedDict(sorted(raster_dict.items()))



#df = joy.geochem_from_csv('../datafiles/geochem_merge_20221026.csv',
df = joy.geochem_from_csv('../datafiles/geochem_merge_20231117.csv',
                          longitude_field_name='Longitude', latitude_field_name='Latitude')

model_dir = '../luffi/REM_surfaces_csv/'
gc_interpolator_dict = joy.make_gc_interpolator_dict(model_dir)


region = [-80,-50,-40,-10]
present_day_topography = pygmt.datasets.load_earth_relief(resolution='02m', region=region)


boschman_polygons = gpd.read_file('/Users/simon/GIT/bx/andes_paper/boschman/reconstruction_model/reconstructed_0.00Ma.shp')

#boschman_polygons.plot()

In [ ]:
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle

def make_error_boxes(ax, xdata, ydata, xerror, yerror, facecolor='r',
                     edgecolor='None', alpha=0.5):

    # Create list for all the error patches
    errorboxes = []

    # Loop over data points; create box from errors at each point
    for x, y, xe, ye in zip(xdata, ydata, xerror.T, yerror.T):
        rect = Rectangle((x - xe, y - ye), xe*2, ye*2)
        errorboxes.append(rect)

    # Create patch collection with specified colour/alpha
    pc = PatchCollection(errorboxes, facecolor=facecolor, alpha=alpha,
                         edgecolor=edgecolor)

    # Add collection to axes
    ax.add_collection(pc)

    
def add_error_boxes(ax, subset):

    for method in method_dictionary:
        subsubset = subset[subset['Method Category']==method]
        x=(subsubset['Age_min']+subset['Age_max'])/2
        y=(subsubset['Elevation_min']+subset['Elevation_max'])/2
        xerr=(subset['Age_max']-subset['Age_min'])/2
        yerr=(subset['Elevation_max']-subset['Elevation_min'])/2

        ax.plot(x, y, '.', color=method_dictionary[method], label=method)
        ax.errorbar(x, y, xerr=xerr, yerr=yerr, elinewidth=3, alpha=0.5,
                    fmt='.', color=method_dictionary[method])

        make_error_boxes(ax, np.array(x), np.array(y), np.array(xerr), np.array(yerr), 
                         facecolor=method_dictionary[method], alpha=0.1)
    
    
    
df_Boschman = pd.read_excel('~/OneDrive/Andes_works/datafiles/Boschman_compilation.xlsx', sheet_name='Sheet1')

df_Boschman.dropna(subset=['Age_max'], inplace=True)

df_Boschman['Method Category'].fillna(value='miscellaneous', inplace=True)

method_dictionary = {'stratigraphy': 'orange', 
                     'miscellaneous': 'purple', 
                     'isotope paleoaltimetry': 'blue',
                     'paleobotany/fossil leaf physiognomy': 'green'}

print(df_Boschman.Domain.unique())
print(df_Boschman.Polygon_ID.unique())

In [ ]:
age_max = 50
time_bin_size = 5.
pygmt.config(GMT_VERBOSE='e')

fid_list = [
    ('GPlates-7ee2ab84-8574-4532-b63a-90bada6a1d19', 'WC1'), #2516, W Cordillera NN 
    ('GPlates-24d8ed85-d32e-4e66-b03f-fd5b176d2c6a', 'A1'), #2595,  Altiplano N 

    ('GPlates-79c86152-d456-445c-9639-ad48e9cae130', 'PCF1'), #2598, Coastal N 
    ('GPlates-cc221f9a-b81c-4590-929d-7ff88905fe6d', 'WC2'), #2573, W Cordillera N
    ('GPlates-23a60a2d-39b5-4346-a260-24e3dc960c94', 'A2'), #2527,  Altiplano S

    ('GPlates-6b48b619-3774-407a-93e9-f53eadd47137', 'PCF2'), #2526, Coastal C
    ('GPlates-b3771664-dd72-4e88-badb-e92f4e6dd96a', 'WC3'), #2527, W Cordillera C 
    ('GPlates-39186b26-7fab-48cf-9752-f1ac439c1e38', 'P1'), #Puna W

    ('GPlates-22aa7316-c1b2-4940-a0ee-cd1ba8f76311', 'PCF3'), #2592 Coastal S
    ('GPlates-bc8139a3-0f87-4387-a655-432aab84dea6', 'WC4'), #2515, W Cordillera S 
    ('GPlates-f9464e82-d349-402e-8b5e-3df7b2e019a4', 'P2'), #Puna E
           ]


plot_calibrations = [
    ('luffi', 41),
    ('luffi', 'la_yb_elevation'),
    ('Hu', 'la_yb_elevation'),
    ('Hu', 'sr_y_elevation'),
    ('FarnerLee', 'la_yb_elevation'),
    ('FarnerLee', 'gd_yb_elevation')
]


for calibration, mohometer_selection in plot_calibrations:

    if isinstance(mohometer_selection, list):
        mohometer_description_string = '|'.join(mohometer_selection)
    else:
        mohometer_description_string = str(mohometer_selection)


    fig,axs = plt.subplots(nrows=4, ncols=3, figsize=(12,14))

    ps = []
    axs = axs.reshape(-1)
    #for i,(fid,ax) in enumerate(zip(fid_list, np.delete(axs, 0))):
    for i,(fid,ax) in enumerate(zip(fid_list, axs[1:])):

        p = boschman_polygons[boschman_polygons['FEATURE_ID']==fid[0]].iloc[0]
        ps.append(p)

        if p['NAME'] is None:
            p['NAME']=str(p['PLATEID1'])
        print(p['NAME'])
        print(p['FEATURE_ID'])

        df_filt = joy.filter_the_database(df.clip(p.geometry), calibration, 
                                          age_min=-0.1, age_max=age_max, nans_to_zeros=False)
        elevations_df = joy.get_elevations(df_filt, 
                                           gc_interpolator_dict=gc_interpolator_dict,
                                           calibration=calibration,
                                           mohometer_selection=mohometer_selection)
            
        ppdat = gpd.GeoDataFrame(elevations_df, geometry=df_filt.geometry, crs=4326).join(df_filt['age'])
        ppdat['bin_age'] = np.round(ppdat['age']/time_bin_size) * time_bin_size
        time_binned = ppdat.groupby(by=['bin_age'])
        
        median_elevations = []
        elevation_violins = []
        for group in time_binned:
            if not group[1].empty:
                median_elevations.append(group[1].drop(columns=['age', 'bin_age', 'geometry']).stack().median())
                elevation_violins.append(group[1].drop(columns=['age', 'bin_age', 'geometry']).stack())
                group[0]
            else:
                median_elevations.append(np.nan)
                elevation_violins.append([np.nan,np.nan])
        

        bounds = p.geometry.bounds
        try:
            (paleoDEM_stats, 
             paleoDEM_hist) = joymap.get_raster_stats_in_polygon_time_series(boschman_rasters, 
                                                                             boschman_rotation_model, 
                                                                             p,
                                                                             plate_id_to_rotate_to=p.PLATEID1, 
                                                                             old_anchor_plate_id=201,
                                                                             spacing=0.1,
                                                                             region=[bounds[0], bounds[2], bounds[1], bounds[3]])
        except:
            print('Problem extracting raster data')
            continue

        #elevations_df['bin_age'] = np.round(df_filt['age']/time_bin_size) * time_bin_size
        #tmp_cat = []
        #for col in elevations_df.columns:
        #    if 'elevation' in col:
        #        tmp_cat.append(elevations_df[['bin_age', col]].rename(columns={col:'elevation'}).dropna())
        #tmp_cat = pd.concat(tmp_cat, ignore_index=True)
        #positions = tmp_cat['bin_age'].unique()

        ax.computed_zorder = False

        ax.fill_between(x=paleoDEM_stats['age'],
                        y1=paleoDEM_stats['min_elevation'], y2=paleoDEM_stats['max_elevation'], 
                        zorder=0, alpha=0.25, color='grey')
        ax.fill_between(x=paleoDEM_stats['age'],
                        y1=paleoDEM_stats['q25_elevation'], y2=paleoDEM_stats['q75_elevation'], 
                        zorder=0, alpha=0.75, color='grey')
        parts = ax.boxplot(elevation_violins, positions=list(time_binned.groups.keys()), 
                           widths=4, 
                           patch_artist=True, 
                           manage_ticks=False, 
                           #whis=[5, 95], 
                           showfliers=False)
        for pc in parts['boxes']:
            pc.set_facecolor('red')
            pc.set_alpha(0.3)
            pc.set_edgecolor('red')
        for median in parts['medians']:
            median.set_color('red')
            median.set_linewidth(8)
            median.set_solid_capstyle("butt")
        ax.set_xticks(np.arange(0,351,50)/time_bin_size)
        for col in ppdat.columns:
            if 'elevation' in col:
                ax.plot(df_filt.age, ppdat[col], 'o', color='blue', markersize=5, alpha=0.05)
        ax.set_xlim(age_max,0)
        ax.set_ylim(0,6000)
        #ax.set_title('{:s}, {:d}, {:s}'.format(p['NAME'], p['PLATEID1'], ', '.join(['{:0.1f}'.format(i) for i in p.geometry.bounds])))
        #ax.set_title('{:s}'.format(p['NAME']))
        ax.set_title('{:s}'.format(fid[1]))

        if i>7:
            ax.set_xlabel('Reconstruction Time [Ma]')
        else:
            ax.set_xlabel('')
            ax.set_xticklabels('')

        if i in [2,5,8]:
            ax.set_ylabel('Elevation [m]')
        else:
            ax.set_ylabel('')
            ax.set_yticklabels('')

        if fid[0] in df_Boschman.dropna(subset='Polygon_ID').Polygon_ID.unique():
            subset = df_Boschman.query('Polygon_ID == @fid')
            add_error_boxes(ax, subset)

    #boschman_polygons.plot(ax=axs[1])
    ax = axs[0]
    polys = gpd.GeoDataFrame.from_records(ps)
    present_day_topography.plot(ax=ax, zorder=0, cmap='gray_r', vmin=-500, vmax=7000, add_colorbar=False)
    polys['label'] = [x[1] for x in fid_list]
    polys.plot(cmap='Set3', ax=ax, alpha=0.5, edgecolor='white')
    polys.apply(lambda x: ax.annotate(text=x['label'], xy=x.geometry.centroid.coords[0], ha='center', fontsize=8), axis=1);
    ax.set_xlim(-77,-64)
    ax.set_ylim(-28.5,-12)
    ax.set_xlabel('')
    ax.set_ylabel('')

    plt.savefig('../images/Boschman_comparison_{:s}_{:s}.png'.format(calibration, 
                                                                     mohometer_description_string))
    plt.close()
    #plt.show()
